In [1]:
# Load the generative model

include("generative_model.jl")

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##generate_table_world#312", Bool[0], false)

In [2]:
function condition(model, args, list_of_conditions)
    
    all_satisfied = false
    
    world = model(args)
    counter = 0
    while all_satisfied == false
        if all([c(world) for c in list_of_conditions])
            all_satisfied = true
        else
            world = model(args)
        end
        counter += 1
    end
    
    return world, ("NUM_SIMS", counter)
            
end

condition (generic function with 1 method)

In [18]:
default_world_dims = [1000, 1000, 1000]

# scene = generate_table_world(default_world_dims)
# println(scene)

# The table is extremely large
cond1 = w -> (w[1]["dims"][1] > 600) && (w[1]["dims"][3] > 600)

# The table is extremely tall

cond2 = w -> w[1]["dims"][2] > 600

# There are at least 5 blocks
cond3 = w -> length(w) >= 6

@time world = condition(generate_table_world, default_world_dims, [cond1, cond2, cond3])[1]

  0.040457 seconds (37.45 k allocations: 2.103 MiB, 98.84% compilation time)


9-element Vector{Any}:
 Dict{String, Any}("mass" => 100, "base_shape" => ["table"], "abs_spatial" => Dict("x" => 500.0, "z" => 500.0, "y" => 304.0), "dims" => [714, 608, 714], "color" => "white", "object_type" => "table")
 Dict{String, Any}("volume" => 1000, "color" => "yellow", "base_shape" => ["cube"], "abs_spatial" => Dict("x" => 149.67387060369845, "z" => 410.01548092266466, "y" => 613.0), "dims" => [10, 10, 10], "mass" => 1.25, "object_type" => "block")
 Dict{String, Any}("volume" => 4913, "color" => "red", "base_shape" => ["cube"], "abs_spatial" => Dict("x" => 259.80774266631687, "z" => 368.3422705715194, "y" => 616.5), "dims" => [17, 17, 17], "mass" => 6.14125, "object_type" => "block")
 Dict{String, Any}("volume" => 2197, "color" => "yellow", "base_shape" => ["cube"], "abs_spatial" => Dict("x" => 453.50005234141025, "z" => 193.308426267991, "y" => 614.5), "dims" => [13, 13, 13], "mass" => 2.74625, "object_type" => "block")
 Dict{String, Any}("volume" => 2197, "color" => "red", 

In [20]:
function how_many_on_table(world)
    counter = 0
    for i in 2:length(world)
        if world[i]["abs_spatial"]["y"] - world[i]["dims"][2] / 2 == world[1]["dims"][2]
            counter += 1
        end
    end
    return counter
end

how_many_on_table (generic function with 1 method)

In [4]:
# How many blocks are there?

w -> length(filter_type("block", w))

# How many blocks are on the table?

w -> length(filter_type("block", filter_preposition((on, "table"), w)))

# How many red blocks are on the table?
    
w -> length(filter_color("red", filter_type("block", filter_preposition((on, "table"), w))))

# Are there more yellow blocks or red blocks?

num_yellow = length(filter_color("yellow", filter_type("block", w)))
num_red = length(filter_color("red", filter_type("block", w)))
w -> num_yellow > num_red ? "yellow blocks" : (num_yellow < num_red ? "red blocks" : "same")

# What is the color of the heaviest block?

red_blocks = filter_color("red", filter_type("block", w))
w -> the_color_of(max_by("mass", red_blocks))



